In [ ]:
import phd

import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

# Some plotting niceties
phd.plots.setup_ncc()

# Ensure TIMIT is extracted
timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

# Some general params
utt = 'wash'
spkr = 'CAG0'

In [ ]:
%%javascript
// This adds a button to the toolbar to reset the kernel
// and run the first cell. Handy for iterating.
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = spkr
samples = timit.word_samples([utt])
model = phd.sermo.AuditoryFeatures()
model.audio = samples[utt][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(x, x, zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("MFCC")
ax2.set_ylabel("MFCC (z-scored)")
ax2.set_xlabel("Frame")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-mfcc')

In [ ]:
# MFCCs with dt=0.001
model.mfcc.dt = 0.001
x = model.mfcc()
print x.shape

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(x, x, zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("MFCC")
ax2.set_ylabel("MFCC (z-scored)")
ax2.set_xlabel("Frame")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-mfcc-long')

In [ ]:
# Get NCCs
net = model.build(nengo.Network(seed=0))
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(sim.data[c_p], sim.data[c_p], zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("NCC")
ax2.set_ylabel("NCC (z-scored)")
ax2.set_xticklabels(sim.trange())
ax2.set_xlabel("Time (s)")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-ncc')

In [ ]:
# NCCs shortened to 31 frames
n_frames = 31
ncc = phd.experiments.shorten(sim.data[c_p], n_frames=n_frames)

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(ncc, ncc, zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("NCC")
ax2.set_ylabel("NCC (z-scored)")
ax2.set_xticklabels(sim.trange(dt=0.01))
ax2.set_xlabel("Time (s)")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-ncc-short')

In [ ]:
# A peek into the neural implementation
fig = plt.figure(figsize=(6, 10))

ax = plt.subplot(3, 1, 1)
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs, ax=ax, cbar=False)
ax.set_title("Inner hair cell activity")
ax.set_xticks(())

ax = plt.subplot(3, 1, 2)
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs, ax=ax, cbar=False)
ax.set_title("IHC-AN synaptic activity")
ax.set_xticks(())

ax = plt.subplot(3, 1, 3)
rasterplot(sim.trange(), sim.data[an_p])
ax.set_title("Spiking AN neural activity")
ax.set_ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles)
ax.set_ylabel("Neuron")
ax.set_xlabel("Time (s)")
sns.despine()

fig.tight_layout()
phd.plots.savefig(fig, 'results', 'ncc-periphery')

## With derivatives

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = spkr
samples = timit.word_samples([utt])
model = phd.sermo.AuditoryFeatures()
model.audio = samples[utt][0]
model.add_derivative('IntermediateDeriv')  # First derivative
model.add_derivative('FeedforwardDeriv')  # Second derivative

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(x, x, zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("MFCC")
ax2.set_ylabel("MFCC (z-scored)")
ax2.set_xlabel("Frame")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-mfcc-derivs')

In [ ]:
# Get NCCs
net = model.build(nengo.Network(seed=0))
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(sim.data[c_p], sim.data[c_p], zscore=(False, True))
f.suptitle("Features for utterance of '%s'" % utt, fontsize='large')
ax1.set_ylabel("NCC")
ax2.set_ylabel("NCC (z-scored)")
ax2.set_xticklabels(sim.trange())
ax2.set_xlabel("Time (s)")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'results', 'ncc-ncc-derivs')

## Running a classification experiment

In [ ]:
# Not a full experiment, just showing how testing / training works
# with a small data set.
model = phd.sermo.AuditoryFeatures()
expt = phd.experiments.AuditoryFeaturesExperiment(
    model, phones=phd.timit.TIMIT.consonants)
expt.seed = 0
expt.timit.filefilt.region = 8
expt.timit.filefilt.sex = 'F'
expt.timit.filefilt.sent_type = 'I'
key = expt.run()
res = phd.experiments.AuditoryFeaturesResult.load(key)
print("MFCC training acc: %f" % res.mfcc_train_acc)
print("NCC training acc: %f" % res.ncc_train_acc)
print("MFCC testing acc: %f" % res.mfcc_test_acc)
print("NCC testing acc: %f" % res.ncc_test_acc)

## Plotting experimental results

In [ ]:
pargs = {'columns': [], 'vary': 'phones', 'filter_by': [], 'hue_order': ["vowels", "consonants", "all"]}
phd.plots.ncc_accuracy(relative=False, **pargs)
phd.plots.ncc_accuracy(relative=True, **pargs)
phd.plots.ncc_time(**pargs)

In [ ]:
phd.plots.ncc_accuracy(['zscore'], 'zscore', hue_order=['False', 'True'], relative=False)

In [ ]:
phd.plots.ncc_accuracy(relative=False, columns=['derivatives'], vary='derivatives', hue_order=['0', '1', '2'])

In [ ]:
phd.plots.ncc_accuracy(['derivtype'], 'derivtype',
                       hue_order=['FeedforwardDeriv', 'IntermediateDeriv'])

In [ ]:
phd.plots.ncc_tsaccuracy(['periphery'], 'periphery')

In [ ]:
phd.plots.ncc_tsaccuracy(['feature'], 'feature')
phd.plots.ncc_time(['feature'], 'feature', [str(x) for x in [1, 8, 12, 16, 32, 64]])

In [ ]:
phd.plots.ncc_accuracy(
    relative=False, columns=['dt'], vary='dt', hue_order=["0.010000", "0.005000", "0.001000"])

In [ ]:
columns = ['periphmodel', 'adaptive']
porder = ['gammatone', 'log_gammachirp', 'dual_resonance', 'compressive_gammachirp', 'tan_carney']
phd.plots.ncc_accuracy(columns, 'periphmodel', hue_order=porder, relative=True)
phd.plots.ncc_time(columns, 'periphmodel', hue_order=porder)
phd.plots.ncc_accuracy(columns, 'adaptive', hue_order=['False', 'True'], relative=True)

## Scaling

In [ ]:
def n_neurons(msg, model):
    net = model.build()
    nn = sum(e.n_neurons for e in net.all_ensembles)
    print("=== %s ===" % msg)
    pneurons = model.freqs.size * model.periphery.neurons_per_freq
    print("Periphery layer: %d freqs x %d neurons_per_freq = %d neurons" % (
        model.freqs.size, model.periphery.neurons_per_freq, pneurons))
    cneurons = model.n_cepstra * model.cepstra.n_neurons
    print("Feature layer: %d cepstra x %d neurons_per_cepstra = %d neurons" % (
        model.n_cepstra, model.cepstra.n_neurons, cneurons))
    for i, deriv in enumerate(model.derivatives):
        # Note! Assumes FeedforwardDeriv!
        cneurons += model.n_cepstra * 2 * deriv.n_neurons
        print("Derivative %d: %d cepstra x 2 x %d neurons_per_cepstra = %d neurons" % (
            (i+1), model.n_cepstra, deriv.n_neurons, model.n_cepstra * deriv.n_neurons))

    assert pneurons + cneurons == nn
    print("Total: %d neurons" % nn)
    print("")

model = phd.sermo.AuditoryFeatures()
model.add_derivative()
n_neurons("Default configuration", model)
model = phd.sermo.AuditoryFeatures()
model.freqs = phd.filters.erbspace(20, 20000, 3500)
model.periphery.neurons_per_freq = 20
model.n_cepstra = 20
model.cepstra.n_neurons = 50
model.add_derivative(n_neurons=50)
n_neurons("Conservative estimate", model)
model = phd.sermo.AuditoryFeatures()
model.freqs = phd.filters.erbspace(20, 20000, 3500)
model.periphery.neurons_per_freq = 40
model.n_cepstra = 40
model.cepstra.n_neurons = 200
model.add_derivative(n_neurons=200)
model.add_derivative(n_neurons=200)
n_neurons("Generous estimate", model);